In [60]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output

import base64
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AACPY10 import AnimalShelter



###########################
# Data Manipulation / Model
###########################

shelter = AnimalShelter()


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image = "Grazioso Salvare Logo.png"
encoded_img = base64.b64encode(open(image, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_img.decode()))
    ),
    html.P('Made by Mitch Sfakianos'),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value ='Water'
        ),
        html.Div(id='dd-output-container')
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
     html.Hr(),
    # insert chart and map into site
     html.Div(className='row',
              style={'display': 'flex'},
              children=[
                  html.Div(
                      id='map-id',
                      className='col s12 m6'
                  ),
                  dcc.Graph(
                      id='pie-chart',
                      className='col s12 m6'
                  )
              ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# filter options
@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    if (value == 'water'):
        df = pd.DataFrame(list(shelter.read({
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": { "$gte": 26.0 },
            "$and": [{
                "age_upon_outcome_in_weeks": { "$lte": 156.0 }
            }]
        })))
    elif (value == 'mountain'):
        df = pd.DataFrame(list(shelter.read({
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": { "$gte": 26.0 },
            "$and": [{
                "age_upon_outcome_in_weeks": { "$lte": 156.0 }
            }]
        })))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.read({
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Doberman Pinscher",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": { "$gte": 20.0 },
            "$and": [{
                "age_upon_outcome_in_weeks": { "$lte": 300.0 }
            }]
        })))
    else:
        df = pd.DataFrame.from_records(shelter.read(None))
    return df.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#pie chart
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('datatable-id', "data")])
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    chart = px.pie(dff, names='breed')
    return chart


# map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    dff = df if viewData is None else pd.DataFrame(viewData)
    
    choice = None
    if selected_rows:
        choice = dff.iloc[selected_rows[0]]
    else:
        choice = dff.iloc[0]
              
    breed = choice[3]
    name = choice[8]
    lat = choice[12]
    long = choice[13]
    
    return[
        dl.Map(style={'width':'1000px', 'height':'500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, long], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]


app